In [1]:
import subprocess
import osmium
import re

import geopandas as gpd
import pandas as pd

from geopy.geocoders import Nominatim

In [59]:
LEVEL_HEIGHT = 3.4

# https://wiki.openstreetmap.org/wiki/Simple_3D_buildings#Other_roof_tags
def _feet_to_meters(s):
    r = re.compile("([0-9]*\.?[0-9]+)'([0-9]*\.?[0-9]+)?\"?")
    m = r.findall(s)[0]
    if len(m[0]) > 0 and len(m[1]) > 0:
        m = float(m[0]) + float(m[1]) / 12.0
    elif len(m[0]) > 0:
        m = float(m[0])
    return m * 0.3048

def _get_height(tags):
    if 'height' in tags:
        # already accounts for roof
        if '\'' in tags['height'] or '\"' in tags['height']:
            return _feet_to_meters(tags['height'])
        r = re.compile(r"[-+]?\d*\.\d+|\d+")
        return float(r.findall(tags['height'])[0])
    if 'levels' in tags:
        roof_height = 0
        if 'roof_height' in tags:
            if '\'' in tags['roof_height'] or '\"' in tags['roof_height']:
                roof_height = _feet_to_meters(tags['roof_height'])
            else:
                r = re.compile(r"[-+]?\d*\.\d+|\d+")
                roof_height = float(r.findall(tags['roof_height'])[0])

        # does not account for roof height
        height = float(tags['levels']) * LEVEL_HEIGHT
        if 'roof_levels' in tags and roof_height == 0:
            height += float(tags['roof_levels']) * LEVEL_HEIGHT
        return height
    return 7.0

def _get_min_height(tags):
    if 'min_height' in tags:
        # already accounts for roof
        if '\'' in tags['min_height'] or '\"' in tags['min_height']:
            return _feet_to_meters(tags['min_height'])
        r = re.compile(r"[-+]?\d*\.\d+|\d+")
        return float(r.findall(tags['min_height'])[0])
    if 'min_level' in tags:
        height = float(tags['min_level']) * LEVEL_HEIGHT
        return height
    return 0.0

class BuildingHandler(osmium.SimpleHandler):

    def __init__(self):
        osmium.SimpleHandler.__init__(self)
        self.geometry = []
        self.height = []
        self.min_height = []
        self.wkbfab = osmium.geom.WKBFactory()

    def get_gdf(self):
        geometry = gpd.GeoSeries.from_wkb(self.geometry, crs='epsg:4326')
        height = pd.Series(self.height, dtype='float')
        min_height = pd.Series(self.min_height, dtype='float')
        
        return gpd.GeoDataFrame({
            'geometry': geometry,
            'min_height': self.min_height,
            'height': self.height
        }, index=geometry.index)

    def area(self, a):
        tags = a.tags
        # Qualifiers
        if not ('building' in tags or 'building:part' in tags or tags.get('type', None) == 'building'):
            return
        # Disqualifiers
        if (tags.get('location', None) == 'underground' or 'bridge' in tags):
            return
        try:
            poly = self.wkbfab.create_multipolygon(a)
            self.geometry.append(poly)
            self.height.append(_get_height(tags))
            self.min_height.append(_get_min_height(tags))
        except Exception as e:
            print(e)
            print(a)

In [67]:
city_full = 'Washington DC'
city = 'dc'
filename = 'data/osm/%s.osm.pbf'%(city)
input_filename = 'data/osm/north-america-latest.osm.pbf'

geolocator = Nominatim(user_agent='uic')
location = geolocator.geocode(city_full).raw
print(location)

bbox = [float(x) for x in location['boundingbox']]
bbox = [{'lat': bbox[0], 'lon': bbox[2]}, {'lat': bbox[1], 'lon': bbox[3]}] # bottom left, top right

aux = '%f,%f,%f,%f'%(bbox[0]['lon'],bbox[0]['lat'],bbox[1]['lon'],bbox[1]['lat'])
proc = subprocess.call(['wsl', 'osmium-tool/build/osmium', 'extract', '-b', aux, '-o', filename, '--overwrite', input_filename], shell=True)

{'place_id': 283012477, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'relation', 'osm_id': 5396194, 'boundingbox': ['38.7916303', '38.995968', '-77.1197949', '-76.909366'], 'lat': '38.8950368', 'lon': '-77.0365427', 'display_name': 'Washington, District of Columbia, United States', 'class': 'boundary', 'type': 'administrative', 'importance': 0.8592888986115819, 'icon': 'https://nominatim.openstreetmap.org/ui/mapicons//poi_boundary_administrative.p.20.png'}


In [68]:
h = BuildingHandler()
h.apply_file(filename, locations=True)

In [69]:
gdf = h.get_gdf()
gdf = gdf.to_crs('epsg:3395')
gdf.to_feather('data/osm/%s.feather'%city, compression='lz4')  

C:\Users\fabio\AppData\Local\Temp\ipykernel_30420\2113808017.py:3: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  gdf.to_feather('data/osm/%s.feather'%city, compression='lz4')
